### Imports

In [8]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [9]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [10]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [11]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [12]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [13]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-74    2.5   16     0.91   0        Ruim     0.99      
-44    1.5   19     0.08   1        Bom      1.00      
-57    1.1   9      0.24   1        Bom      0.99      
-68    4.7   85     0.83   0        Ruim     1.00      
-46    1.0   16     0.31   1        Bom      1.00      
-64    4.2   98     0.89   0        Ruim     1.00      
-75    10.4  63     0.74   0        Ruim     1.00      
-64    11.7  65     0.80   0        Ruim     1.00      
-74    15.0  76     0.88   0        Ruim     1.00      
-89    3.2   49     0.75   0        Ruim     1.00      
-69    5.6   48     0.74   0        Ruim     1.00      
-62    4.1   101    0.60   0        Ruim     1.00      
-70    1.8   8      0.53   1        Bom      0.85      
-52    1.2   39     0.18   1        Bom      0.99      
-54    1.6   12     0.10   1        Bom      1.00      
-57    2.2   11     0.10   1        Bom      0.9

### Save

In [14]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
